In [11]:
import os
import sys
src_path = sys.path[0].replace("exp_notebook", "src")
#replace notebook as scripts
data_path = sys.path[0].replace("exp_notebook", "data")
if src_path not in sys.path:
    sys.path.append(src_path)
out_path = sys.path[0].replace("exp_notebook", "output")
#sys.path.append("..")

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import copy

from grid_manager import MPC_op
from data_loader import UCSD_dataloader
from battery_model import Battery_base
from optimizer import Battery_optimizer
from predictor import *
from exp_manager import ExperimentManager
from threading import Thread
from time import sleep,ctime

# from ev_manager import EVmanager
# import copy

In [12]:
bat_params_sample ={
    "bat_capacity": None,
    "bat_p_max": 3, # i.e., capacity (kWh) / p_bat_max (kW) = 3 (h)
    "bat_p_min": 3, # can omit, then p_bat_min = p_bat_max
    "bat_price": 150, # $/kWh (old: 1000, ref: Tesla Powerwall)
    "bat_efficacy": 0.98, 
    "bat_life_0": 3650, # days.
    "bat_cycle_0": 3000, # cycles in lifetime
    # battery degradation params
    "deg_model": "throughput",  
        # valid values: "throughput", "Crate", "rainflow", "DOD"
    #   [1. degradation ~ high C-rate]
    "deg_Crate_thres": (0.25, 0.25, 0.25, 0.25),
    "deg_Crate_lambda": (0.8, 1, 1.5, 2),
    #   [2. degradation ~ large cycle depth]
    "deg_rainflow_thres": (0.2, 0.2, 0.2, 0.4),
    "deg_rainflow_lambda": (0.6, 1, 1.5, 1.8),
    #   [3. degradation ~ low SoE range]
    "deg_DOD_thres": (0.5, 0.2, 0.2, 0.1),
    "deg_DOD_lambda": (1.3, 1.15, 0.85, 0.6),
}

op_params_sample = {"K": 96,
            "dc_price": 0.6,
            "ev_efficacy": 0.98,
            "energy_price_sell": 0.6, 
            "deg_model_opt": "unconscious",
            "ev_charge_rule": "flex",
            "ev_charge_rule_default": "unif",
            "p_grid_max": "1.5",}

# week based bat 6h varied dc

In [13]:
exp_suffix = "Aug2023_debug3"
exp_folder = os.path.join(out_path, "experiments", exp_suffix)
debug_folder = os.path.join(out_path, "debug_test")
assert os.path.exists(exp_folder)
log_fn = os.path.join(exp_folder, "debug.xlsx")#_oneday_12months
save_path = os.path.join(exp_folder, "MPC-debug")

In [14]:
'''
class MPC_ExperimentManager(ExperimentManager):
    """
    Main variable: battery size (normalized to hr: to bld_load)
    """
    def run_one_trial(self, params, save_fn):
        
        # params: keys: "pred_model", "strategy", "B_kWh", "deg_model_opt", "deg_model", 
        #   "start", "end", "p_grid_max", "price_dc", "price_sell", "ev_charge_rule"

        op_params = copy.deepcopy(op_params_sample)
        bat_params = copy.deepcopy(bat_params_sample)

        bat_params["deg_model"] = params.get("deg_model", "DOD")
        bat_params["bat_capacity"] = params.get("B_kWh", 350)
        strategy = params.get("strategy", "optimal")
        op_params["deg_model_opt"] = params.get("deg_model_opt", "rainflow") 
        op_params["ev_charge_rule"] = params.get("ev_charge_rule", "flex")
        op_params["energy_price_sell"] = params.get("price_sell", "rainflow")
        op_params["dc_price"] = params.get("price_dc", 0.6) 
        op_params["penalty_coef"]=params.get("penalty_coef", 0)
        op_params["dc_formulation"]=params.get("dc_formulation", "moving")
        p_grid_max = params.get("p_grid_max", "1.5") 
        op_params["p_grid_max"] = None if p_grid_max is None else str(p_grid_max)
        pred_model = params.get("pred_model", "GT") 
        
        # 2023/05/30 LunLong
        # Trying to add bld pv ev into grid search parmas
        bld=params.get("bld", "Hopkins")
        pv=params.get("pv", "Hopkins")
        ev=params.get("ev","OSLER")   
        
        # 2023/07/21 LunLong
        # Check params sanity with XGBoost prediction added
        #if params.get("pred_model")=='Prediction':
        #    if not params.get("bld")=="Sum" & params.get("pv")=='Sum':
        #        raise Exception("Incompatible pred_model and bld/pv setting.")
        
        if bld is None or pv is None or ev is None:
            raise Exception("Building related params are not prepoperly initiated")

        def convert_time(s):
            idx_hyphen = s.index("-")
            month, day = int(s[:idx_hyphen]), int(s[idx_hyphen+1:])
            return datetime(2019, month, day, 0, 0)
        t_start = convert_time(params.get("start", "10-1"))
        t_end = convert_time(params.get("end", "10-8"))

        mpc = MPC_op()

        # Step 1: load data
        mpc.load_data(loader=UCSD_dataloader, 
            tstart=datetime(2019,1,1,0,0), tend=datetime(2020,1,1,0,0), delta=0.25,
            bld=bld, pv=pv, ev=ev, pv_to_bld=0.5, ev_to_bld=0.25, Pmax=10)

        # Step 2: Load historical data
        mpc.init_historical_data(loader=UCSD_dataloader,
            tstart=datetime(2018,1,1,0,0), tend=datetime(2019,12,31,23,59), delta=0.25,
            bld=bld, pv=pv, ev=ev, pv_to_bld=0.5, ev_to_bld=0.25, Pmax=10)

        # Step 3: specify other operational params
        optimizer_params = {"strategy": strategy, "language":"gurobi"}

        mpc.init_op_params(optimizer=Battery_optimizer, optimizer_params=optimizer_params, delta_0=0.25, **op_params)

        # Step 4: specify battery
        mpc.init_battery(model=Battery_base, params=bat_params, delta_0=0.25)

        # Step 5: initialize predictor
        # [Yi, 2023/03/08] modify predictor def
        mpc.init_predictor(shortcut=pred_model)

        # Step 6: initialize save_config
        mpc.init_save_config(save_fn=save_fn[:-5],  # FIXME: remove ".xlsx"
            folder_path=self.save_path, log_pred_action=True,
            checkpoints="1D", recovery=True, recovery_from=None,
            )

        mpc.run(tstart=t_start, tend=t_end, exe_K=4, save=True)
        
        stats = dict(mpc.summary["All"])
        
        return stats


em = MPC_ExperimentManager(log_fn=log_fn, save_path=save_path, save=True, exp_prefix="MPC")
var_keys = [
            "method", "strategy",
            "pred_model", "deg_model_opt", "p_grid_max", 
            "price_dc", "price_sell", 
            "ev_charge_rule", 
            "B_kWh",  "deg_model", 
            "start", "end", 
            "bld", "ev", "pv",
            "dc_formulation","penalty_coef"]
em.run(keys=var_keys, num_trials=10)'''

'\nclass MPC_ExperimentManager(ExperimentManager):\n    """\n    Main variable: battery size (normalized to hr: to bld_load)\n    """\n    def run_one_trial(self, params, save_fn):\n        \n        # params: keys: "pred_model", "strategy", "B_kWh", "deg_model_opt", "deg_model", \n        #   "start", "end", "p_grid_max", "price_dc", "price_sell", "ev_charge_rule"\n\n        op_params = copy.deepcopy(op_params_sample)\n        bat_params = copy.deepcopy(bat_params_sample)\n\n        bat_params["deg_model"] = params.get("deg_model", "DOD")\n        bat_params["bat_capacity"] = params.get("B_kWh", 350)\n        strategy = params.get("strategy", "optimal")\n        op_params["deg_model_opt"] = params.get("deg_model_opt", "rainflow") \n        op_params["ev_charge_rule"] = params.get("ev_charge_rule", "flex")\n        op_params["energy_price_sell"] = params.get("price_sell", "rainflow")\n        op_params["dc_price"] = params.get("price_dc", 0.6) \n        op_params["penalty_coef"]=params

In [15]:
'''
class MPC_ExperimentManager(ExperimentManager):
    """
    Main variable: battery size (normalized to hr: to bld_load)
    """
    def run_one_trial(self, params, save_fn):
        
        # params: keys: "pred_model", "strategy", "B_kWh", "deg_model_opt", "deg_model", 
        #   "start", "end", "p_grid_max", "price_dc", "price_sell", "ev_charge_rule"

        op_params = copy.deepcopy(op_params_sample)
        bat_params = copy.deepcopy(bat_params_sample)

        bat_params["deg_model"] = params.get("deg_model", "DOD")
        bat_params["bat_capacity"] = params.get("B_kWh", 350)
        strategy = params.get("strategy", "optimal")
        op_params["deg_model_opt"] = params.get("deg_model_opt", "rainflow") 
        op_params["ev_charge_rule"] = params.get("ev_charge_rule", "flex")
        op_params["energy_price_sell"] = params.get("price_sell", "rainflow")
        op_params["dc_price"] = params.get("price_dc", 0.6) 
        p_grid_max = params.get("p_grid_max", "1.5") 
        op_params["penalty_coef"]=params.get("penalty_coef", 0)
        op_params["dc_formulation"]=params.get("dc_formulation", "moving"),
        op_params["sol_save_steps"] = params.get("sol_save_steps",0)
        op_params["p_grid_max"] = None if p_grid_max is None else str(p_grid_max)
        pred_model = params.get("pred_model", "GT") 
        
        # 2023/05/30 LunLong
        # Trying to add bld pv ev into grid search parmas
        bld=params.get("bld", "Hopkins")
        pv=params.get("pv", "Hopkins")
        ev=params.get("ev","OSLER")   
        
        # 2023/07/21 LunLong
        # Check params sanity with XGBoost prediction added
        #if params.get("pred_model")=='Prediction':
        #    if not params.get("bld")=="Sum" & params.get("pv")=='Sum':
        #        raise Exception("Incompatible pred_model and bld/pv setting.")
        
        if bld is None or pv is None or ev is None:
            raise Exception("Building related params are not prepoperly initiated")

        def convert_time(s):
            """
            idx_hyphen = s.index("-")
            month, day, hour, minute = int(s[:idx_hyphen]), int(s[idx_hyphen+1:idx_hyphen+2]),\
                int(s[idx_hyphen+2:idx_hyphen+3]),int(s[idx_hyphen+3:])
            """
            t_l=s.split("-")
            month,day,hour,minute=int(t_l[0]),int(t_l[1]),int(t_l[2]),int(t_l[3]) 
            
            return datetime(2019, month, day, hour, minute)
        t_start = convert_time(params.get("start", "10-1-0-0"))
        t_end = convert_time(params.get("end", "10-8-0-0"))

        mpc = MPC_op()

        # Step 1: load data
        mpc.load_data(loader=UCSD_dataloader, 
            tstart=datetime(2019,1,1,0,0), tend=datetime(2020,1,1,0,0), delta=0.25,
            bld=bld, pv=pv, ev=ev, pv_to_bld=0.5, ev_to_bld=0.25, Pmax=10)

        # Step 2: Load historical data
        mpc.init_historical_data(loader=UCSD_dataloader,
            tstart=datetime(2018,1,1,0,0), tend=datetime(2019,12,31,23,59), delta=0.25,
            bld=bld, pv=pv, ev=ev, pv_to_bld=0.5, ev_to_bld=0.25, Pmax=10)

        # Step 3: specify other operational params
        optimizer_params = {"strategy": strategy, "language":"gurobi"}

        mpc.init_op_params(optimizer=Battery_optimizer, optimizer_params=optimizer_params, delta_0=0.25, **op_params)

        # Step 4: specify battery
        mpc.init_battery(model=Battery_base, params=bat_params, delta_0=0.25)

        # Step 5: initialize predictor
        # [Yi, 2023/03/08] modify predictor def
        mpc.init_predictor(shortcut=pred_model)

        # Step 6: initialize save_config
        mpc.init_save_config(save_fn=save_fn[:-5],  # FIXME: remove ".xlsx"
            folder_path=self.save_path, log_pred_action=True,
            checkpoints="1D", recovery=True, recovery_from=None,
            )

        mpc.run(tstart=t_start, tend=t_end, exe_K=4, save=True)
        
        stats = dict(mpc.summary["All"])
        
        return stats'''

'\nclass MPC_ExperimentManager(ExperimentManager):\n    """\n    Main variable: battery size (normalized to hr: to bld_load)\n    """\n    def run_one_trial(self, params, save_fn):\n        \n        # params: keys: "pred_model", "strategy", "B_kWh", "deg_model_opt", "deg_model", \n        #   "start", "end", "p_grid_max", "price_dc", "price_sell", "ev_charge_rule"\n\n        op_params = copy.deepcopy(op_params_sample)\n        bat_params = copy.deepcopy(bat_params_sample)\n\n        bat_params["deg_model"] = params.get("deg_model", "DOD")\n        bat_params["bat_capacity"] = params.get("B_kWh", 350)\n        strategy = params.get("strategy", "optimal")\n        op_params["deg_model_opt"] = params.get("deg_model_opt", "rainflow") \n        op_params["ev_charge_rule"] = params.get("ev_charge_rule", "flex")\n        op_params["energy_price_sell"] = params.get("price_sell", "rainflow")\n        op_params["dc_price"] = params.get("price_dc", 0.6) \n        p_grid_max = params.get("p_grid_

In [16]:
'''em = MPC_ExperimentManager(log_fn=log_fn, save_path=save_path, save=True, exp_prefix="MPC")
var_keys = [
            "method", "strategy",
            "pred_model", "deg_model_opt", "p_grid_max", 
            "price_dc", "price_sell", 
            "ev_charge_rule", 
            "B_kWh",  "deg_model", 
            "start", "end", 
            "bld", "ev", "pv",
            "dc_formulation","penalty_coef",
            "sol_save_steps"]
em.run(keys=var_keys, num_trials=1)'''

'em = MPC_ExperimentManager(log_fn=log_fn, save_path=save_path, save=True, exp_prefix="MPC")\nvar_keys = [\n            "method", "strategy",\n            "pred_model", "deg_model_opt", "p_grid_max", \n            "price_dc", "price_sell", \n            "ev_charge_rule", \n            "B_kWh",  "deg_model", \n            "start", "end", \n            "bld", "ev", "pv",\n            "dc_formulation","penalty_coef",\n            "sol_save_steps"]\nem.run(keys=var_keys, num_trials=1)'

In [17]:
'''{'bat_capacity': 438, 
 'p_grid': {0: array([ 55.0967    ,  53.4154    ,  54.4315    ,  54.7357    ,
        53.4565    ,  53.3265    ,  53.1925    ,  53.6792    ,
       168.0258    , 168.0258    , 168.0258    , 168.0258    ,
       168.0258    , 168.0258    , 168.0258    , 168.0258    ,
         0.        ,   0.        ,   0.        ,  52.88197303,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
        69.21336785,  28.04075802,  22.79872744,  30.16683692,
        26.47498135,   3.0355011 ,   0.        ,   0.        ,
       147.9836775 , 135.63361998,  26.07661927,  58.04937128,
       168.0258    , 168.0258    , 168.0258    , 168.0258    ,
       168.0258    , 168.0258    , 168.0258    , 168.0258    ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
       -22.13698545, -11.51580309,  -2.75138288,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
       -85.76497989, -84.10236905, -80.1134351 , -46.81593349,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
        64.2248    ,  57.5456    ,  55.3379    ,  55.2161    ])},
 'bat_p': {0: array([   0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
       -114.7819    , -114.4962    , -114.5284    , -114.2634    ,
       -113.9955    , -113.9046    , -113.4835    , -114.1572    ,
         54.1552    ,   54.3732    ,   55.0541    ,    2.29442697,
         56.0282    ,   57.3499    ,   58.9026    ,   59.3513    ,
         59.9128    ,   66.4512    ,   65.31404964,   64.14296804,
         62.10344636,   61.14227006,   54.54950405,   46.30655116,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,  -14.99566531,  -30.72518778,
       -146.        , -146.        ,    0.        ,  -75.8857023 ,
       -146.        , -146.        , -146.        , -146.        ,
       -146.        , -146.        , -146.        , -146.        ,
        -91.46057633,  -60.60959131,    0.        ,    0.        ,
          0.        ,    0.        ,   -3.05909547,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
        -51.01928353,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    6.11986794,
         16.40071854,   25.98404131,   36.16849898,   47.1537144 ,
        146.        ,  146.        ,  146.        ,  146.        ,
         70.61576524,  100.8723    ,  104.3886    ,  103.2896    ,
         97.3343936 ,  101.0092    ,   81.7313    ,   71.1124    ,
         70.7388    ,   67.2198    ,   67.0372    ,   65.7838    ,
          0.        ,    0.        ,    0.        ,    0.        ])}, 
 'bat_e': {0: array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,  28.1215655 ,  56.1731345 ,  84.2325925 ,
       112.2271255 , 140.156023  , 168.06265   , 195.8661075 ,
       223.8346215 , 210.01951946, 196.14880517, 182.10439191,
       181.5190789 , 167.22617074, 152.59609421, 137.56992074,
       122.42928299, 107.14540544,  90.1935687 ,  73.53182134,
        57.16881929,  41.32610338,  25.72858551,  11.8128957 ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   3.673938  ,
        11.20160901,  46.97160901,  82.74160901,  82.74160901,
       101.33360607, 137.10360607, 172.87360607, 208.64360607,
       244.41360607, 280.18360607, 315.95360607, 351.72360607,
       387.49360607, 409.90144727, 424.75079714, 424.75079714,
       424.75079714, 424.75079714, 424.75079714, 425.50027553,
       425.50027553, 425.50027553, 425.50027553, 425.50027553,
       425.50027553, 438.        , 438.        , 438.        ,
       438.        , 438.        , 438.        , 438.        ,
       436.4388092 , 432.25495243, 425.62637046, 416.39971256,
       404.37070379, 367.12580583, 329.88090787, 292.63600991,
       255.39111195, 237.37688612, 211.64415653, 185.01441163,
       158.66502388, 133.83482143, 108.06716837,  87.21734694,
        69.07642857,  51.03081633,  33.88290816,  16.78158163,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ])}, 
 'ev_p': {0: array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])}, 
 'ev_e': {0: array([[ 0.   ,  0.   ,  0.   , ..., 12.065, 12.065, 12.065],
       [ 0.   ,  0.   ,  0.   , ..., 12.065, 12.065, 12.065],
       [ 0.   ,  0.   ,  0.   , ..., 12.065, 12.065, 12.065],
       ...,
       [ 0.   ,  0.   ,  0.   , ..., 15.748, 15.748, 15.748],
       [ 0.   ,  0.   ,  0.   , ..., 15.748, 15.748, 15.748],
       [ 0.   ,  0.   ,  0.   , ..., 15.748, 15.748, 15.748]])}, 
 'eq_cycles': array([0.])}'''

"{'bat_capacity': 438, \n 'p_grid': {0: array([ 55.0967    ,  53.4154    ,  54.4315    ,  54.7357    ,\n        53.4565    ,  53.3265    ,  53.1925    ,  53.6792    ,\n       168.0258    , 168.0258    , 168.0258    , 168.0258    ,\n       168.0258    , 168.0258    , 168.0258    , 168.0258    ,\n         0.        ,   0.        ,   0.        ,  52.88197303,\n         0.        ,   0.        ,   0.        ,   0.        ,\n         0.        ,   0.        ,   0.        ,   0.        ,\n         0.        ,   0.        ,   0.        ,   0.        ,\n        69.21336785,  28.04075802,  22.79872744,  30.16683692,\n        26.47498135,   3.0355011 ,   0.        ,   0.        ,\n       147.9836775 , 135.63361998,  26.07661927,  58.04937128,\n       168.0258    , 168.0258    , 168.0258    , 168.0258    ,\n       168.0258    , 168.0258    , 168.0258    , 168.0258    ,\n         0.        ,   0.        ,   0.        ,   0.        ,\n         0.        ,   0.        ,   0.        ,   0.        ,\n

In [18]:
class MPC_ExperimentManager(ExperimentManager):
    """
    Main variable: battery size (normalized to hr: to bld_load)
    """
    def run_one_trial(self, params, save_fn):
        
        # params: keys: "pred_model", "strategy", "B_kWh", "deg_model_opt", "deg_model", 
        #   "start", "end", "p_grid_max", "price_dc", "price_sell", "ev_charge_rule"

        op_params = copy.deepcopy(op_params_sample)
        bat_params = copy.deepcopy(bat_params_sample)

        bat_params["deg_model"] = params.get("deg_model", "DOD")
        bat_params["bat_capacity"] = params.get("B_kWh", 350)
        strategy = params.get("strategy", "optimal")
        op_params["deg_model_opt"] = params.get("deg_model_opt", "rainflow") 
        op_params["ev_charge_rule"] = params.get("ev_charge_rule", "flex")
        op_params["energy_price_sell"] = params.get("price_sell", "rainflow")
        op_params["dc_price"] = params.get("price_dc", 0.6) 
        p_grid_max = params.get("p_grid_max", "1.5") 
        op_params["penalty_coef"]=params.get("penalty_coef", 0)
        op_params["sol_save_steps"] = params.get("sol_save_steps",0)
        op_params["dc_formulation"]=params.get("dc_formulation", "moving")
        op_params["p_grid_max"] = None if p_grid_max is None else str(p_grid_max)
        pred_model = params.get("pred_model", "GT") 
        
        # 2023/05/30 LunLong
        # Trying to add bld pv ev into grid search parmas
        bld=params.get("bld", "Hopkins")
        pv=params.get("pv", "Hopkins")
        ev=params.get("ev","OSLER")   
        
        # 2023/07/21 LunLong
        # Check params sanity with XGBoost prediction added
        #if params.get("pred_model")=='Prediction':
        #    if not params.get("bld")=="Sum" & params.get("pv")=='Sum':
        #        raise Exception("Incompatible pred_model and bld/pv setting.")
        
        if bld is None or pv is None or ev is None:
            raise Exception("Building related params are not prepoperly initiated")

        def convert_time(s):
            s_l=s.split("-")
            
            month, day ,hour, minute=int(s_l[0]),int(s_l[1]) ,int(s_l[2]) ,int(s_l[3]) 
            return datetime(2019, month, day, hour, minute)
        t_start = convert_time(params.get("start", "10-1-0-0"))
        t_end = convert_time(params.get("end", "10-8-0-0"))

        mpc = MPC_op()

        # Step 1: load data
        mpc.load_data(loader=UCSD_dataloader, 
            tstart=datetime(2019,1,1,0,0), tend=datetime(2020,1,1,0,0), delta=0.25,
            bld=bld, pv=pv, ev=ev, pv_to_bld=0.5, ev_to_bld=0.25, Pmax=10)

        # Step 2: Load historical data
        mpc.init_historical_data(loader=UCSD_dataloader,
            tstart=datetime(2018,1,1,0,0), tend=datetime(2019,12,31,23,59), delta=0.25,
            bld=bld, pv=pv, ev=ev, pv_to_bld=0.5, ev_to_bld=0.25, Pmax=10)

        # Step 3: specify other operational params
        optimizer_params = {"strategy": strategy, "language":"gurobi"}

        mpc.init_op_params(optimizer=Battery_optimizer, optimizer_params=optimizer_params, delta_0=0.25, **op_params)

        # Step 4: specify battery
        mpc.init_battery(model=Battery_base, params=bat_params, delta_0=0.25)

        # Step 5: initialize predictor
        # [Yi, 2023/03/08] modify predictor def
        mpc.init_predictor(shortcut=pred_model)

        # Step 6: initialize save_config
        mpc.init_save_config(save_fn=save_fn[:-5],  # FIXME: remove ".xlsx"
            folder_path=self.save_path, log_pred_action=True,
            checkpoints="1D", recovery=True, recovery_from=None,
            )

        mpc.run(tstart=t_start, tend=t_end, exe_K=4, save=True)
        
        stats = dict(mpc.summary["All"])
        
        return stats

em = MPC_ExperimentManager(log_fn=log_fn, save_path=save_path, save=True, exp_prefix="MPC")
var_keys = [
            "method", "strategy",
            "pred_model", "deg_model_opt", "p_grid_max", 
            "price_dc", "price_sell", 
            "ev_charge_rule", 
            "B_kWh",  "deg_model", 
            "start", "end", 
            "bld", "ev", "pv",
            "dc_formulation","penalty_coef",
            "sol_save_steps"]
em.run(keys=var_keys, num_trials=4)


!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 0 sessions
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 0 sessions
========================= 2019-06-01 00:00:00 =========================
========================= 2019-06-01 01:00:00 =========================
========================= 2019-06-01 02:00:00 =========================
========================= 2019-06-01 03:00:00 =========================
========================= 2019-06-01 04:00:00 =========================
========================= 2019-06-01 05:00:00 =========================
========================= 2019-06-01 06:00:00 =========================
========================= 2019-06-01 07:00:00 =========================
========================= 2019-06-01 08:00:00 =========================
========================= 2019-06-01 09:00:00 =========================
========================= 2019-06-01 10:00:00 =========================
========================= 2019-06-01 11:00:00 =========================
==================

In [19]:
def exp():
    #print(args)
    #sleep(10)
    class MPC_ExperimentManager(ExperimentManager):
        """
        Main variable: battery size (normalized to hr: to bld_load)
        """
        def run_one_trial(self, params, save_fn):
            
            # params: keys: "pred_model", "strategy", "B_kWh", "deg_model_opt", "deg_model", 
            #   "start", "end", "p_grid_max", "price_dc", "price_sell", "ev_charge_rule"

            op_params = copy.deepcopy(op_params_sample)
            bat_params = copy.deepcopy(bat_params_sample)

            bat_params["deg_model"] = params.get("deg_model", "DOD")
            bat_params["bat_capacity"] = params.get("B_kWh", 350)
            strategy = params.get("strategy", "optimal")
            op_params["deg_model_opt"] = params.get("deg_model_opt", "rainflow") 
            op_params["ev_charge_rule"] = params.get("ev_charge_rule", "flex")
            op_params["energy_price_sell"] = params.get("price_sell", "rainflow")
            op_params["dc_price"] = params.get("price_dc", 0.6) 
            p_grid_max = params.get("p_grid_max", "1.5") 
            op_params["penalty_coef"]=params.get("penalty_coef", 0)
            op_params["sol_save_steps"] = params.get("sol_save_steps",0)
            op_params["dc_formulation"]=params.get("dc_formulation", "moving")
            op_params["p_grid_max"] = None if p_grid_max is None else str(p_grid_max)
            pred_model = params.get("pred_model", "GT") 
            
            # 2023/05/30 LunLong
            # Trying to add bld pv ev into grid search parmas
            bld=params.get("bld", "Hopkins")
            pv=params.get("pv", "Hopkins")
            ev=params.get("ev","OSLER")   
            
            # 2023/07/21 LunLong
            # Check params sanity with XGBoost prediction added
            #if params.get("pred_model")=='Prediction':
            #    if not params.get("bld")=="Sum" & params.get("pv")=='Sum':
            #        raise Exception("Incompatible pred_model and bld/pv setting.")
            
            if bld is None or pv is None or ev is None:
                raise Exception("Building related params are not prepoperly initiated")

            def convert_time(s):
                s_l=s.split("-")
                
                month, day ,hour, minute=int(s_l[0]),int(s_l[1]) ,int(s_l[2]) ,int(s_l[3]) 
                return datetime(2019, month, day, hour, minute)
            t_start = convert_time(params.get("start", "10-1-0-0"))
            t_end = convert_time(params.get("end", "10-8-0-0"))

            mpc = MPC_op()

            # Step 1: load data
            mpc.load_data(loader=UCSD_dataloader, 
                tstart=datetime(2019,1,1,0,0), tend=datetime(2020,1,1,0,0), delta=0.25,
                bld=bld, pv=pv, ev=ev, pv_to_bld=0.5, ev_to_bld=0.25, Pmax=10)

            # Step 2: Load historical data
            mpc.init_historical_data(loader=UCSD_dataloader,
                tstart=datetime(2018,1,1,0,0), tend=datetime(2019,12,31,23,59), delta=0.25,
                bld=bld, pv=pv, ev=ev, pv_to_bld=0.5, ev_to_bld=0.25, Pmax=10)

            # Step 3: specify other operational params
            optimizer_params = {"strategy": strategy, "language":"gurobi"}

            mpc.init_op_params(optimizer=Battery_optimizer, optimizer_params=optimizer_params, delta_0=0.25, **op_params)

            # Step 4: specify battery
            mpc.init_battery(model=Battery_base, params=bat_params, delta_0=0.25)

            # Step 5: initialize predictor
            # [Yi, 2023/03/08] modify predictor def
            mpc.init_predictor(shortcut=pred_model)

            # Step 6: initialize save_config
            mpc.init_save_config(save_fn=save_fn[:-5],  # FIXME: remove ".xlsx"
                folder_path=self.save_path, log_pred_action=True,
                checkpoints="1D", recovery=True, recovery_from=None,
                )

            mpc.run(tstart=t_start, tend=t_end, exe_K=4, save=True)
            
            stats = dict(mpc.summary["All"])
            
            return stats
    for i in range(100):
        try:
            em = MPC_ExperimentManager(log_fn=log_fn, save_path=save_path, save=True, exp_prefix="MPC")
            var_keys = [
                        "method", "strategy",
                        "pred_model", "deg_model_opt", "p_grid_max", 
                        "price_dc", "price_sell", 
                        "ev_charge_rule", 
                        "B_kWh",  "deg_model", 
                        "start", "end", 
                        "bld", "ev", "pv",
                        "dc_formulation","penalty_coef",
                        "sol_save_steps"]
            em.run(keys=var_keys, num_trials=100)
        except Exception as e:
            print('Reason_out:', e)  
            print('Failed in '+str(i)+'nd trial')
            continue
        
t1=Thread(target=exp)
t2=Thread(target=exp)
t3=Thread(target=exp)
#t4=Thread(target=exp)
#t5=Thread(target=exp)

t1.start()
t2.start()
t3.start()
#t4.start()
#t5.start()

t1.join()
t2.join()
t3.join()
#t4.join()
#t5.join()

Access denied, suspending.
Access denied, suspending.
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 0 sessions
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 0 sessions
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 0 sessions
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 0 sessions
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 0 sessions
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 0 sessions
========================= 2019-06-01 00:00:00 =========================
params["dc_prev_max"]: [0]
0.15
========================= 2019-06-01 01:00:00 =========================
params["dc_prev_max"]: [65.4195]
0.15
========================= 2019-06-01 00:00:00 =========================
params["dc_prev_max"]: [0]
0.2
========================= 2019-06-01 02:00:00 =========================
params["dc_prev_max"]: [114.73508558771601]
0.15
========================= 2019-06-01 01:00:00 =========================
params["dc_prev_max"]: [65.4195]
0.2
========================= 2019-06-01 03:00:00

In [20]:
'''def parma_generator():
    key_dic={
        "status":	,
        "bld":	,
        "pv":	,
        "ev":	,
        "method":	,
        "strategy":	,
        "B_kWh":	,
        "pred_model":	,
        "deg_model_opt":	,
        "deg_model":	,
        "start":	,
        "end":	,
        'week_of_year':	,
        "p_grid_max":	,
        "price_dc":	,
        "dc_formulation":	,
        "penalty_coef":	,
        "price_sell":	,
        "ev_charge_rule":	,
        "sol_save_steps":

    }
    '''

'def parma_generator():\n    key_dic={\n        "status":\t,\n        "bld":\t,\n        "pv":\t,\n        "ev":\t,\n        "method":\t,\n        "strategy":\t,\n        "B_kWh":\t,\n        "pred_model":\t,\n        "deg_model_opt":\t,\n        "deg_model":\t,\n        "start":\t,\n        "end":\t,\n        \'week_of_year\':\t,\n        "p_grid_max":\t,\n        "price_dc":\t,\n        "dc_formulation":\t,\n        "penalty_coef":\t,\n        "price_sell":\t,\n        "ev_charge_rule":\t,\n        "sol_save_steps":\n\n    }\n    '